In [2]:
import dspy
from dotenv import load_dotenv

load_dotenv()

mini = dspy.OpenAI(model = "gpt-4o-mini", max_tokens = 4000)
print(mini("Say hello"))

['Hello! How can I assist you today?']


In [4]:
dspy.settings.configure(lm = mini)

In [5]:
class AgentDesigner(dspy.Signature):
    """Your task is to design Agents for the application domain in the provided Python framework."""

    application_domain = dspy.InputField()
    agent_framework_description = dspy.InputField()
    #discovered_agent_archive = dspy.InputField()
    optimal_agent = dspy.OutputField(desc = "IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!")

In [6]:
agent_designer = dspy.ChainOfThought(AgentDesigner)

In [7]:
dspy_framework_description = """
You will code the agent with the DSPy Python framework.

Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question.

class Agent(dspy.Module):
    def __init__(self, num_passages = 3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k = num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self, question):
        context = self.retrieve(question).passages
        answer = self.generate_answer(context, question)

        return answer

Here is an example of a blog post writer Agent.

class Agent(dspy.Module):
    def __init__(self):
        self.question_to_blog_outline = dspy.ChainOfThought("question -> blog_outline")
        self.topic_to_paragraph = dspy.ChainOfThought("topic, contexts -> paragraph")
        self.proof_reader = dspy.ChainOfThought("blog_post -> proof_read_blog_post")
        self.title_generator = dspy.ChainOfThought("blog_outline -> title")

    def forward(self, question):
        contexts = dspy.Retrieve(k = 5)(question).passages
        contexts = "".join(contexts)
        raw_blog_outline = self.question_to_blog_outline(question = question, contexts = contexts).blog_outline
        blog_outline = raw_blog_outline.split(',') #Add type hint in expanded Signature
        blog = ""
        for topic in blog_outline:
            topic_contexts = dspy.Retrieve(k = 5)(topic).passages
            topic_contexts = "".join(topic_contexts)
            blog += self.topic_to_paragraph(topic = topic, contexts = topic_contexts).paragraph
            blog += "\n\n"
        blog = self.proof_reader(blog_post = blog).proof_read_blog_post
        title = self.title_generator(blog_outline = raw_blog_outline).title
        final_blog = f'{title} \n\n {blog}'
        return dspy.Prediction(blog = final_blog)

PLEASE NOTE!! It is extremely important that your Agent class is also named "Agent" as shown in the example!! THIS IS EXTREMELY IMPORTANT!!
"""


In [8]:
agent_design = agent_designer(
    application_domain = "Produce Scientific Papers given a repository and a research direction as an input.",
    agent_framework_description = dspy_framework_description
).optimal_agent

print(agent_design)

```python
class Agent(dspy.Module):
    def __init__(self):
        self.retrieve_papers = dspy.Retrieve(k=5)
        self.synthesize_paper = dspy.ChainOfThought("context, research_direction -> scientific_paper")
        self.proof_reader = dspy.ChainOfThought("scientific_paper -> proof_read_scientific_paper")
        self.title_generator = dspy.ChainOfThought("research_direction, context -> title")

    def forward(self, research_direction):
        contexts = self.retrieve_papers(research_direction).passages
        contexts = "".join(contexts)
        raw_scientific_paper = self.synthesize_paper(context=contexts, research_direction=research_direction).scientific_paper
        proof_read_paper = self.proof_reader(scientific_paper=raw_scientific_paper).proof_read_scientific_paper
        title = self.title_generator(research_direction=research_direction, context=contexts).title
        final_paper = f'{title}\n\n{proof_read_paper}'
        return dspy.Prediction(scientific_paper=final

In [10]:
mini.inspect_history(n=1)




Your task is to design Agents for the application domain in the provided Python framework.

---

Follow the following format.

Application Domain: ${application_domain}

Agent Framework Description: ${agent_framework_description}

Reasoning: Let's think step by step in order to ${produce the optimal_agent}. We ...

Optimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!

---

Application Domain: Produce Scientific Papers given a repository and a research direction as an input.

Agent Framework Description: You will code the agent with the DSPy Python framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages = 3): super().__init__() self.retrieve = dspy.Retrieve(k = num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, question): context = self

'\n\n\nYour task is to design Agents for the application domain in the provided Python framework.\n\n---\n\nFollow the following format.\n\nApplication Domain: ${application_domain}\n\nAgent Framework Description: ${agent_framework_description}\n\nReasoning: Let\'s think step by step in order to ${produce the optimal_agent}. We ...\n\nOptimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!\n\n---\n\nApplication Domain: Produce Scientific Papers given a repository and a research direction as an input.\n\nAgent Framework Description: You will code the agent with the DSPy Python framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages = 3): super().__init__() self.retrieve = dspy.Retrieve(k = num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, qu

In [11]:
agent_design = agent_designer(
    application_domain="Create a comprehensive travel itinerary given a destination and duration of stay.",
    agent_framework_description=dspy_framework_description
).optimal_agent

print(agent_design)

```python
class Agent(dspy.Module):
    def __init__(self):
        self.retrieve_info = dspy.Retrieve(k=5)
        self.itinerary_generator = dspy.ChainOfThought("destination, duration, contexts -> itinerary")
        self.formatter = dspy.ChainOfThought("itinerary -> formatted_itinerary")

    def forward(self, destination, duration):
        contexts = self.retrieve_info(destination).passages
        itinerary = self.itinerary_generator(destination=destination, duration=duration, contexts=contexts).itinerary
        formatted_itinerary = self.formatter(itinerary=itinerary).formatted_itinerary
        return dspy.Prediction(itinerary=formatted_itinerary)
```


In [12]:
mini.inspect_history(n=1)




Your task is to design Agents for the application domain in the provided Python framework.

---

Follow the following format.

Application Domain: ${application_domain}

Agent Framework Description: ${agent_framework_description}

Reasoning: Let's think step by step in order to ${produce the optimal_agent}. We ...

Optimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!

---

Application Domain: Create a comprehensive travel itinerary given a destination and duration of stay.

Agent Framework Description: You will code the agent with the DSPy Python framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages = 3): super().__init__() self.retrieve = dspy.Retrieve(k = num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, question): context = self.

'\n\n\nYour task is to design Agents for the application domain in the provided Python framework.\n\n---\n\nFollow the following format.\n\nApplication Domain: ${application_domain}\n\nAgent Framework Description: ${agent_framework_description}\n\nReasoning: Let\'s think step by step in order to ${produce the optimal_agent}. We ...\n\nOptimal Agent: IMPORTANT!! ONLY OUTPUT THE CODE FOR THE AGENT IN THE PROVIDED PYTHON FRAMEWORK!!\n\n---\n\nApplication Domain: Create a comprehensive travel itinerary given a destination and duration of stay.\n\nAgent Framework Description: You will code the agent with the DSPy Python framework. Here is an example of defining a program that retrieves information from a knowledge source and then uses the context to answer a question. class Agent(dspy.Module): def __init__(self, num_passages = 3): super().__init__() self.retrieve = dspy.Retrieve(k = num_passages) self.generate_answer = dspy.ChainOfThought("context, question -> answer") def forward(self, que